In [8]:
import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

# the following line is not required if BatchFlow is installed as a python package.
sys.path.append('../../../batchflow')

from batchflow import B, V, F, R, P, C, Dataset, Batch, action, inbatch_parallel, ImagesBatch, Pipeline
from batchflow.models.torch import TorchModel
from batchflow.models.metrics import ClassificationMetrics
from batchflow.opensets import MNIST
from batchflow import best_practice

In [9]:
BATCH_SIZE = 64
mnist = MNIST()

Extracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/train-images-idx3-ubyte.gz
Downloading
 http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
ExtractingExtracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/t10k-labels-idx1-ubyte.gz
 /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/t10k-images-idx3-ubyte.gz


In [10]:
from batchflow.models.torch.layers import Dense

class MyModel(TorchModel):
    @classmethod
    def default_config(cls):
        config = TorchModel.default_config()
        config['head'].update(dict(units=10, layout='f'))
        return config

    def build_config(self, names=None):
        config = super().build_config(names)
        config['head/units'] = self.num_classes('targets')
        return config

    @classmethod
    def body(cls, **kwargs):
        kwargs = cls.get_defaults('body', kwargs)
        x = Dense(**kwargs)
        return x

In [11]:
model_config = {
    'inputs': dict(images={'shape': (28, 28, 1)},
                   labels={'classes': 10}),
    'loss': 'ce',
    'optimizer': 'Adam',
    'initial_block/inputs': 'images',
    'head/units': 10,
    'output': dict(predicted=['proba', 'labels'])
}

In [12]:
train_pipeline = (mnist.p
    .init_variable('loss_history', init_on_each_run=list)
    .init_model('dynamic', MyModel, 'my_model', model_config)
    .train_model('my_model', B('images'), B('labels'),
                 fetches='loss', save_to=V('loss_history'), mode='a'))

In [17]:
train_pipeline.run(BATCH_SIZE, shuffle=True, n_epochs=1, bar=True)

AttributeError: 'NoneType' object has no attribute 'gen_batch'